### Modules and constants

In [ ]:
import os

%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
from matplotlib import colors

VECTORDIM = 3
TENSORSQRTDIM = 3
TENSORDIM = 9
DEVSYMTENSORDIM = 5
DEVSYMTENSOR_INDEX = [0,1,2,4,5]

nscalar_invariants = 2
nbasis_tensors = 4

### Case specifics and plotting options

In [ ]:
tinit = 0
tfinal = 100
data_dir = 'data'

results_dir = 'results' 
foam_dir = 'foam_primal' 

mesh_shape = [50, 50]
flow = 0

scaleUx = 0.5
scaleUy = 500.0
scaleUz = 500.0

lim_option = 'same'  # ['different', 'same']
subfigsize = 3
cmap = 'Spectral'
save_figures = False
save_fig_dir = 'figures' 
fig_ext = 'png'
contour = False

### Physics functions

In [ ]:
def get_tensor_basis(gradU, time_scale):
    ncells = len(time_scale)
    T = np.zeros([ncells, DEVSYMTENSORDIM, nbasis_tensors])
    for i, (igradU, it) in enumerate(zip(gradU, time_scale)):
        igradU = igradU.reshape([TENSORSQRTDIM, TENSORSQRTDIM])
        S = it * 0.5*(igradU + igradU.T)
        R = it * 0.5*(igradU - igradU.T)
        T1 = S
        T2 = S @ R - R @ S
        T3 = minus_thirdtrace(S @ S)
        T4 = minus_thirdtrace(R @ R)
        for j, iT in enumerate([T1, T2, T3, T4]):
            iT = iT.reshape([TENSORDIM])
            T[i, :, j] = iT[DEVSYMTENSOR_INDEX]
    return T


def minus_thirdtrace(x):
    return x - 1./3.*np.trace(x)*np.eye(TENSORSQRTDIM)


def shih_quadratic(theta):
    def g1(theta):
        num = -2./3.
        denom = 1.25 + np.sqrt(2 * theta[:, 0]) + 0.9 * np.sqrt(-2 * theta[:, 1])
        return num/denom

    def g234(theta, coeff):
        return coeff / (1000. + (2 * theta[:, 0])**(3./2.))

    g = np.empty([len(theta), nbasis_tensors])
    g[:, 0] = g1(theta)
    for i, c in enumerate([7.5, 1.5, -9.5]):
        g[:, i+1] = g234(theta, c)
    return g


def get_b(g, gradU, time_scale):
    T = get_tensor_basis(gradU, time_scale)
    return _get_b(g, T)


def _get_b(g, T):
    return np.sum(np.expand_dims(g,1) * T, axis=-1)


def b2a(b, k):
    return 2 * np.expand_dims(np.squeeze(k), -1) * b

### Plotting functions

In [ ]:
def plot_comp(y, z, val, col_names, row_names, cmap='coolwarm', contour=False, lim_option='same', norm=True):
    Nrows = len(row_names)
    Ncols = len(col_names)
    lim = []
    if lim_option=='same':
        for i in range(Nrows):
            if norm:
                lim.append((min(-np.finfo(float).eps, np.min(val[-1][:, i])), max(np.finfo(float).eps, np.max(val[-1][:, i]))))
            else:
                lim.append((np.min(val[-1][:, i]), np.max(val[-1][:, i])))
    fig, axs = plt.subplots(Nrows, Ncols, figsize=((Ncols+0.5)*subfigsize, (Nrows+0.5)*subfigsize))
    for irow in range (Nrows):
        for icol in range(Ncols): 
            if contour:
                if lim_option=='same':
                    divnorm = colors.TwoSlopeNorm(vcenter=0, vmin=lim[irow][0], vmax=lim[irow][1])
                    cs = axs[irow, icol].contourf(rs(y), rs(z), rs(val[icol][:,irow]), cmap=cmap, vmin=lim[irow][0], vmax=lim[irow][1])
                elif lim_option=='different': 
                    cs = axs[irow, icol].contourf(rs(y), rs(z), rs(val[icol][:,irow]), cmap=cmap)
                    cbar = fig.colorbar(cs, ax=axs[irow, icol], shrink=0.95)
            else:
                if lim_option=='same':
                    if norm:
                        divnorm = colors.TwoSlopeNorm(vcenter=0, vmin=lim[irow][0], vmax=lim[irow][1])
                        cs = axs[irow,icol].pcolor(rs(y), rs(z), rs(val[icol][:,irow]), cmap=cmap, norm=divnorm, shading='auto')
                    else:
                        cs = axs[irow,icol].pcolor(rs(y), rs(z), rs(val[icol][:,irow]), cmap=cmap, vmin=lim[irow][0], vmax=lim[irow][1], shading='auto')
                elif lim_option=='different': 
                    cs = axs[irow,icol].pcolor(rs(y), rs(z), rs(val[icol][:,irow]), cmap=cmap, norm=norm, shading='auto')
                    cbar = fig.colorbar(cs, ax=axs[irow, icol], shrink=0.75)
            plt.setp(axs[irow,icol].get_xticklabels(), visible=False)
            plt.setp(axs[irow,icol].get_yticklabels(), visible=False)
            axs[irow,icol].tick_params(axis='both', which='both', length=0)
            if icol==0:
                axs[irow,icol].set_ylabel(row_names[irow])
            if irow==0:
                axs[irow,icol].set_title(col_names[icol])
            axs[irow, icol].set_aspect('equal', 'box')
        if not contour and lim_option=='same':
            cbar = fig.colorbar(cs, ax=axs[irow], shrink=0.75)
    return fig, axs


def rs(x):
    return x.reshape(mesh_shape)


### Load data

In [ ]:
# coordinates
x = np.loadtxt(os.path.join(data_dir, 'x'))
y = np.loadtxt(os.path.join(data_dir, 'y'))
z = np.loadtxt(os.path.join(data_dir, 'z'))
ncells = mesh_shape[0] * mesh_shape[1]

# # cost
J = []
for i in range(tinit, tfinal+1):
    iJ = np.loadtxt(os.path.join(results_dir, f'J.{i}'))
    J.append(iJ.tolist())

# initial
g_0 = np.loadtxt(os.path.join(results_dir, f'g.{tinit}')) 
if nbasis_tensors==1:
    g_0 = np.expand_dims(np.squeeze(g_0), -1)
U_0 = np.loadtxt(os.path.join(results_dir, f'U.flow_{flow}.{tinit}'))
gradU_0 = np.loadtxt(os.path.join(results_dir, f'gradU.flow_{flow}.{tinit}'))
time_scale_0 = np.loadtxt(os.path.join(results_dir, f'timeScale.flow_{flow}.{tinit}'))
tke_0 = np.loadtxt(os.path.join(results_dir, f'tke.flow_{flow}.{tinit}'))
b_0 = get_b(g_0, gradU_0, time_scale_0)
a_0 = b2a(b_0, tke_0)
W_0 = np.loadtxt(os.path.join(results_dir, f'w.{tinit}'))

# final
g_1 = np.loadtxt(os.path.join(results_dir, f'g.{tfinal}')) 
if nbasis_tensors==1:
    g_0 = np.expand_dims(np.squeeze(g_0), -1)
U_1 = np.loadtxt(os.path.join(results_dir, f'U.flow_{flow}.{tfinal}'))
gradU_1 = np.loadtxt(os.path.join(results_dir, f'gradU.flow_{flow}.{tfinal}'))
time_scale_1 = np.loadtxt(os.path.join(results_dir, f'timeScale.flow_{flow}.{tfinal}'))
tke_1 = np.loadtxt(os.path.join(results_dir, f'tke.flow_{flow}.{tfinal}'))
b_1 = get_b(g_1, gradU_1, time_scale_1)
a_1 = b2a(b_1, tke_1)
W_1 = np.loadtxt(os.path.join(results_dir, f'w.{tfinal}'))

# truth
theta_t = np.load(os.path.join(data_dir, 'scalar_invariants.npy'))[:, :nscalar_invariants]
g_t = shih_quadratic(theta_t)
U_t = np.zeros([ncells, VECTORDIM])
for i, x in enumerate(['x', 'y', 'z']):
    U_t[:, i] = np.loadtxt(os.path.join(data_dir, f'U{x}FullField'))
T_t = np.load(os.path.join(data_dir, 'basis_tensors.npy'))[:, :, :nbasis_tensors]
tke_t = np.load(os.path.join(data_dir, 'tke.npy'))
b_t = _get_b(g_t, T_t)
a_t = b2a(b_t, tke_t)

def get_theta_tilde(gradU, time_scale):
    ncells = len(time_scale)
    theta_tilde = np.zeros([ncells, 1])
    theta = np.zeros([ncells, 1])
    for i, (igradU, it) in enumerate(zip(gradU, time_scale)):
        igradU = igradU.reshape([TENSORSQRTDIM, TENSORSQRTDIM])
        theta_tilde[i] = it**2 * 0.5 * (igradU[1,0]**2 + igradU[2,0]**2)
        S = it * 0.5*(igradU + igradU.T)
        S2 = S @ S
        theta[i] = np.trace(S2)
    return theta_tilde, theta

### Verify $\theta_1\approx\tilde{\theta}$

In [ ]:
# theta_tilde_1, theta_1 = get_theta_tilde(gradU_1, time_scale_1)
# plt.figure()
# plt.plot(theta_1[:,0], theta_tilde_1, 'o')
# # plt.plot(theta_1[:,0], theta_t[:,0], 'o')
# plt.show()


# Cost

In [ ]:
fig, ax = plt.subplots()
ax.plot(J[:tfinal+1], '-')
ax.set_xlabel(r'iteration')
ax.set_ylabel(r'$J$')
ax.set_title(r'Cost $J$')
ax.set_ylim(0., np.max(J[:tfinal+1]))
ax.set_xlim(0., tfinal)

    
fig, ax = plt.subplots()
ax.semilogy(J[:tfinal+1], '-')
ax.set_xlabel(r'iteration')
ax.set_ylabel(r'$J$')
ax.set_title(r'Cost $J$')
limlow = 10**np.floor(np.log10(np.min(J[:tfinal+1])))
ax.set_ylim(limlow, np.max(J[:tfinal+1]))
ax.set_xlim(0., tfinal);

# Weights

In [ ]:
# plt.figure()
# plt.title(r'Weights $W$')
# plt.xlabel(r'Initial Weight')
# plt.ylabel(r'Final Weight')
# line = [min(np.min(W_0), np.min(W_1)), max(np.max(W_0), np.max(W_1))]
# plt.plot(line, [0., 0.], 'k-', alpha=0.5)
# plt.plot([0., 0.], line, 'k-', alpha=0.5)
# plt.plot(W_0, W_1, 'ko')
# plt.plot(line, line, 'y-', alpha=0.5)
# plt.xlim(line)

## $g$ functions

In [ ]:
# plt.figure()
# # plt.plot(theta_t[:, 0], theta_1[:, 0], 'o', color='tab:grey')
# plt.plot(g_t[:, 0], g_1[:, 0], 'o', color='tab:grey')
# lim = [np.min(g_t[:, 0]), np.max(g_t[:, 0])]
# plt.plot(lim, lim, 'k-')

In [ ]:
def g_inplane(g):
    return g[:, 1] - 0.5*g[:, 2] + 0.5*g[:, 3]

g_0n = np.zeros([ncells, 2])
g_0n[:, 0] = g_0[:, 0]
g_0n[:, 1] = g_inplane(g_0)

g_1n = np.zeros([ncells, 2])
g_1n[:, 0] = g_1[:, 0]
g_1n[:, 1] = g_inplane(g_1)

g_tn = np.zeros([ncells, 2])
g_tn[:, 0] = g_t[:, 0]
g_tn[:, 1] = g_inplane(g_t)

col_names = ['Initial', 'Final', 'True']
row_names = [r'$g^{(1)}$', r'$g^{(2)} - 0.5g^{(3)} + 0.5g^{(4)}$']
values = [g_0n, g_1n, g_tn]

plot_comp(y, z, values, col_names, row_names, cmap=cmap, contour=False, lim_option=lim_option, norm=False)

## Velocity

In [ ]:
contour = False
cmap = 'Spectral'
row_names = [r'$u_x$', r'$u_y$'] 
col_names = ['Initial', 'Final', 'True']
values = [U_0[:, :2], U_1[:, :2], U_t[:, :2]]
fig, axs = plot_comp(y, z, values, col_names, row_names, cmap=cmap, contour=contour, lim_option=lim_option, norm=True)

In [ ]:
cellid = 5

fig, axs = plt.subplots(1, 3, figsize=(10, 5))

axs[0].plot(rs(U_t[:, 0]*scaleUx)[:,cellid], rs(y)[cellid, :], 'k-')
axs[0].plot(rs(U_0[:, 0]*scaleUx)[:,cellid], rs(y)[cellid, :], 'r--')
axs[0].plot(rs(U_1[:, 0]*scaleUx)[:,cellid], rs(y)[cellid, :], 'b--')
axs[0].set_ylabel('$x_2$')
axs[0].set_xlabel('$U_1$')


axs[1].plot(rs(U_t[:, 1]*scaleUy)[:,cellid], rs(y)[cellid, :], 'k-')
axs[1].plot(rs(U_0[:, 1]*scaleUy)[:,cellid], rs(y)[cellid, :], 'r--')
axs[1].plot(rs(U_1[:, 1]*scaleUy)[:,cellid], rs(y)[cellid, :], 'b--')
axs[1].set_xlabel('$1000~U_2$')

axs[2].plot(rs(U_t[:, 2]*scaleUz)[:,cellid], rs(y)[cellid, :], 'k-', label='truth')
axs[2].plot(rs(U_0[:, 2]*scaleUz)[:,cellid], rs(y)[cellid, :], 'r--', label='initial')
axs[2].plot(rs(U_1[:, 2]*scaleUz)[:,cellid], rs(y)[cellid, :], 'b--', label='final')
axs[2].set_xlabel('$1000~U_3$')
axs[2].legend()

## Reynolds stress

In [ ]:
cmap = 'Spectral'

Tau_0n = np.zeros([ncells, 3])
Tau_0n[:, 0] = a_0[:, 1]
Tau_0n[:, 1] = a_0[:, 4]
Tau_0n[:, 2] = 2*a_0[:, 3] + a_0[:, 0]

Tau_tn = np.zeros([ncells, 3])
Tau_tn[:, 0] = a_t[:, 1]
Tau_tn[:, 1] = a_t[:, 4]
Tau_tn[:, 2] = 2*a_t[:, 3] + a_t[:, 0]

Tau_1n = np.zeros([ncells, 3])
Tau_1n[:, 0] = a_1[:, 1]
Tau_1n[:, 1] = a_1[:, 4]
Tau_1n[:, 2] = 2*a_1[:, 3] + a_1[:, 0]

row_names = [r'$\tau_{12}$', r'$\tau_{23}$', r'$\tau_{22}-\tau_{33}$']
values = [Tau_0n, Tau_1n, Tau_tn]
fig, axs = plot_comp(y, z, values, col_names, row_names, cmap=cmap, contour=contour, lim_option=lim_option, norm=True)


# For PUBLICATION

In [ ]:
import matplotlib as mpl
# mpl.use("pgf")
mpl.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'figure.autolayout': True,
    'axes.grid': False,
    'axes.labelsize': 10,
    'axes.titlesize': 12,
    'font.size': 10,
    'legend.fontsize': 10,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'savefig.dpi': 500,
    'lines.linewidth': 1.5,
    'lines.markersize': 6,
})

# Figure sizes: JFM text width = 5.3 in, aspect ratio of 1.6 looks good
jfm_one = [4.0, 2.5] # JFM single image
jfm_one_two = [4.5, 5.0] # JFM single image, double height
jfm_two = [2.2, 2.4] # JFM two images


## g

In [ ]:

def g_inplane(g):
    return g[:, 1] - 0.5*g[:, 2] + 0.5*g[:, 3]

thetat = theta_t[:,0]
_, theta0 = get_theta_tilde(gradU_0, time_scale_0)
_, theta1 = get_theta_tilde(gradU_1, time_scale_1)


line_t = {'color': 'tab:grey', 'linestyle': 'none', 'marker': 'o'}
line_0 = {'color': 'tab:red', 'linestyle': 'none', 'marker': 'o', 'markersize': 1}
line_1 = {'color': 'tab:blue', 'linestyle': 'none', 'marker': 'o', 'markersize': 1}

# g1 function vs theta
fig, ax = plt.subplots(1, 1, figsize=jfm_two)
ax.plot(thetat, g_t[:, 0], **line_t)
ax.plot(theta0, g_0[:, 0], **line_0)
ax.plot(theta1, g_1[:, 0], **line_1)
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$g^{(1)}$')
if save_figures:
    plt.savefig(f'{save_fig_dir}/duct_g1.{fig_ext}')

# g2 function vs theta
fig, ax = plt.subplots(1, 1, figsize=jfm_two)
lt = ax.plot(thetat, g_inplane(g_t), **line_t)
l0 = ax.plot(theta0, g_inplane(g_0), **line_0)
l1 = ax.plot(theta1, g_inplane(g_1), **line_1)
ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$g^{(2)} - 0.5g^{(3)} + 0.5g^{(4)}$') 
if save_figures:
    plt.savefig(f'{save_fig_dir}/duct_g2.{fig_ext}')
    
# legend
figlegend = plt.figure(figsize=(3,0.3))
lines = [lt[0], l0[0], l1[0]]
labels = ['Truth', 'Initial', 'Final']
figlegend.legend(lines, labels, 'center', ncol=len(labels))
if save_figures:
    figlegend.savefig(f'{save_fig_dir}/duct_g_legend.{fig_ext}')

## $\theta$

In [ ]:
fig = plt.figure(figsize = jfm_one)
tt = theta_t[:,0].copy()
# N = 5
# tt[tt<=N]=np.nan
# tt[tt>=N]=1.0
divnorm = colors.TwoSlopeNorm(vcenter=0)
cs = plt.pcolor(rs(y), rs(z), rs(tt), cmap=cmap, norm=divnorm, shading='auto')

ax = plt.gca()
plt.setp(ax.get_xticklabels(), visible=False)
plt.setp(ax.get_yticklabels(), visible=False)
ax.tick_params(axis='both', which='both', length=0)
ax.set_aspect('equal', 'box')

if save_figures:
    fig.savefig(f'{save_fig_dir}/duct_theta.{fig_ext}')



fig = plt.figure(figsize=jfm_one)
ax = plt.gca()
plt.colorbar(cs,ax=ax, orientation='vertical', shrink=0.9)
ax.remove()
if save_figures:
    fig.savefig(f'{save_fig_dir}/duct_theta_cb.{fig_ext}', bbox_inches='tight')

## Velocity

In [ ]:
# PLOT VELOCITIES SEPARATELY
cmap = 'Spectral'
names = ['Ux', 'Uy']
ps = ['0', '1', 't']
XY = ['x', 'y']

for i in range(2):
    val = [U_0[:, i], U_1[:, i], U_t[:, i]]
    name = f'duct_{names[i]}'
    vmin = min(-np.finfo(float).eps, np.min(val[-1]))
    vmax = max(np.finfo(float).eps, np.max(val[-1]))
    divnorm = colors.TwoSlopeNorm(vcenter=0, vmin=vmin, vmax=vmax)
    for j, ival in enumerate(val):
        fig = plt.figure(figsize=(2.0, 2.0))
        ax = plt.gca()
        cs = ax.pcolor(rs(y), rs(z), rs(ival), cmap=cmap, norm=divnorm, shading='auto')
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect('equal', 'box')
        if save_figures:
            fig.savefig(f"{save_fig_dir}/{name}_{ps[j]}.{fig_ext}")


    fig = plt.figure(figsize=(2.0, 2.0))
    ax = plt.gca()
    plt.colorbar(cs,ax=ax, orientation='vertical', shrink=0.6)
    ax.remove()
    if save_figures:
        fig.savefig(f'{save_fig_dir}/duct_U{XY[i]}_cb.{fig_ext}',bbox_inches='tight')


# Reynolds Stress

In [ ]:
# PLOT REYNOLDS STRESSES SEPARATELY

names = ['tau12', 'tau23', 'tau22-tau33']
ps = ['0', '1', 't']

for i in range(3):
    name = names[i]
    val = [Tau_0n[:, i], Tau_1n[:, i], Tau_tn[:, i]]
    vmin = min(-np.finfo(float).eps, np.min(val[-1]))
    vmax = max(np.finfo(float).eps, np.max(val[-1]))
    divnorm = colors.TwoSlopeNorm(vcenter=0, vmin=vmin, vmax=vmax)
    for j, ival in enumerate(val):
        fig = plt.figure(figsize=(2.0, 2.0))
        ax = plt.gca()
        cs = ax.pcolor(rs(y), rs(z), rs(ival), cmap=cmap, norm=divnorm, shading='auto')

        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect('equal', 'box')
        if save_figures:
            fig.savefig(f'{save_fig_dir}/duct_{name}_{ps[j]}.{fig_ext}',bbox_inches='tight')


    fig = plt.figure(figsize=(2.0, 2.0))
    ax = plt.gca()
    plt.colorbar(cs,ax=ax, orientation='vertical', shrink=0.6)
    ax.remove()
    if save_figures:
        fig.savefig(f'{save_fig_dir}/duct_{name}_cb.{fig_ext}',bbox_inches='tight')